In [ ]:
from itertools import combinations
from typing import List, Tuple

solver_model_ids = [
    "deepseek-ai/DeepSeek-Prover-V1",
    "Goedel-LM/Goedel-Prover-SFT",
    "ByteDance-Seed/BFS-Prover",
    "AI-MO/Kimina-Prover-Preview-Distill-7B",
    "RickyDeSkywalker/TheoremLlama",
    "wellecks/llmstep-mathlib4-pythia2.8b",
    "internlm/internlm2_5-step-prover",
    "EleutherAI/llemma_7b",
    "kfdong/STP_model_Lean",
    "stoney0062/Leanabell-Prover-GD-RL",
    "deepseek-ai/DeepSeek-Prover-V2-7B",
    "stoney0062/Leanabell-Prover-DS-SFT",
    "deepseek-ai/DeepSeek-Prover-V1.5-RL",
    "ScalableMath/Lean-STaR-plus",
    "kaiyuy/leandojo-lean4-retriever-tacgen-byt5-small",
    "ScalableMath/Lean-STaR-plus",
]

def get_all_combinations_of_length(s: set, length: int) -> List[Tuple]:
    return list(combinations(s, length))

solver_model_ensembles_3 = get_all_combinations_of_length(solver_model_ids, 3)
# assert(len(solver_model_ids) == 15)
# assert(len(solver_model_ensembles_3) == 455)
solver_model_ensembles_3[0]

('deepseek-ai/DeepSeek-Prover-V1',
 'Goedel-LM/Goedel-Prover-SFT',
 'ByteDance-Seed/BFS-Prover')

In [ ]:
pip install --upgrade transformers
!pip install tqdm

In [ ]:
from transformers import pipeline, AutoConfig, AutoTokenizer, AutoModelForCausalLM
import logging

def _load_model(model_id):
    """
    Loads a single model and tokenizer to the GPU, with a fix for rope_scaling issues.
    """
    print(f"Attempting to load model: {model_id}")
    try:
        # 1. Load configuration first
        config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)

        # 3. Load model and tokenizer with the (potentially corrected) config
        tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            config=config, # Pass the corrected config
            torch_dtype="auto",
            trust_remote_code=True
        ).to("cuda")

        print(f"Successfully loaded {model_id}")
        return model, tok

    except Exception as e:
        # Provide a more informative error message
        logging.error(f"❌ Failed to load model '{model_id}'. Error: {e}")
        # Return None to be handled by the calling function, preventing the crash
        return None, None


def generate_proof(pipe, informal_prefix, formal_statement,
                   temperature: float = 0.5, max_new_tokens: int = 4096,
                   num_return_sequences: int = 1):
    prompt = f"{informal_prefix}\n\n{formal_statement}\n\nproof\n"

    # Prepare arguments for the pipeline
    generation_args = {
        'do_sample': True,
        'eos_token_id': pipe.tokenizer.eos_token_id,
        'num_return_sequences': num_return_sequences
    }

    # Only add max_new_tokens if a value is provided
    # If it remains None, the pipeline will use its own default
    if max_new_tokens is not None:
        generation_args['max_new_tokens'] = max_new_tokens

    # Call the pipeline with the arguments
    out = pipe(prompt, **generation_args)

    proofs = [result['generated_text'][len(prompt):].strip() for result in out]
    return proofs

In [ ]:
from pathlib import Path
import gc
import torch
from tqdm import tqdm
from transformers import pipeline

def _sanitize_dir_name(name: str) -> str:
    return str(name).replace("/", "_").replace("\\", "_")

def _next_index(out_dir: Path) -> int:
    out_dir.mkdir(parents=True, exist_ok=True)
    nums = []
    for p in out_dir.glob("*.txt"):
        stem = p.stem
        if stem.isdigit():
            nums.append(int(stem))
    return (max(nums) + 1) if nums else 1

def generate_proofs_memory_safe(
    model_ids,
    problem_row,
    problem_key,                 # e.g., DataFrame index or a unique ID column
    max_attempts: int,
    gpu_batch_size: int = 16,
):
    """
    Generate proofs and write to proofs/<problem_key>/<model_id>/1.txt, 2.txt, ...
    No proof checking; purely generation + IO. Memory-safe (loads one model at a time).
    """
    for model_id in tqdm(model_ids, desc="Models"):
        model = tok = pipe = None
        attempt_bar = None
        try:
            model, tok = _load_model(model_id)
            pipe = pipeline("text-generation", model=model, tokenizer=tok, device=0)

            attempts_left = max_attempts
            attempt_bar = tqdm(total=max_attempts, desc=f"Generating {model_id}", leave=False)

            # Prepare output directory and next index (continues numbering if rerun)
            out_dir = Path("proofs") / _sanitize_dir_name(problem_key) / _sanitize_dir_name(model_id)
            next_idx = _next_index(out_dir)

            print(next_idx)

            # raise Exception("Stop here")

            while attempts_left > 0:
                current_batch_size = min(gpu_batch_size, attempts_left)
                with torch.no_grad():
                    proof_snippets = generate_proof(
                        pipe,
                        informal_prefix=problem_row['informal_prefix'],
                        formal_statement=problem_row['formal_statement'],
                        num_return_sequences=current_batch_size
                    )

                # Write each snippet to numbered files 1.txt, 2.txt, ...
                for snippet in proof_snippets:
                    out_path = out_dir / f"{next_idx}.txt"
                    with open(out_path, "w", encoding="utf-8") as f:
                        f.write(snippet)
                    next_idx += 1

                attempts_left -= current_batch_size
                attempt_bar.update(current_batch_size)

        finally:
            if attempt_bar is not None:
                attempt_bar.close()
            if model: del model
            if tok:   del tok
            if pipe:  del pipe
            gc.collect()
            torch.cuda.empty_cache()

def write_proofs_for_model(
    model_id: str,
    dataframe,
    max_attempts: int = 8,
    gpu_batch_size: int = 8
):
    """
    For each problem in `dataframe`, generate `max_attempts` proofs for `model_id`
    and write them to proofs/<problem_key>/<model_id>/*.txt.
    `problem_key` defaults to the DataFrame index value.
    """
    print(f"--- Generating proofs for model: {model_id} ---")
    for idx, problem_row in tqdm(dataframe.iterrows(), total=len(dataframe), desc=f"Problems for {model_id}"):
        problem_key = problem_row.get('problem_id', idx)  # prefer a column named 'problem_id' if present
        generate_proofs_memory_safe(
            model_ids=[model_id],
            problem_row=problem_row,
            problem_key=problem_key,
            max_attempts=max_attempts,
            gpu_batch_size=gpu_batch_size,
        )
    print(f"--- Done for {model_id}. ---")

In [ ]:
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from datasets import load_dataset
# from google.colab import files
# import time

miniF2F_test_df = load_dataset("AI-MO/minif2f_test", split="train").to_pandas().head(40)

for i, mid in enumerate(solver_model_ids[2:]):
    try:
      write_proofs_for_model(mid, miniF2F_test_df, max_attempts=8, gpu_batch_size=4)
    except Exception as e:
      print(f"Error for {mid}: {e}")

In [ ]:
!zip -r proofs.zip proofs